In [1]:
import pandas as pd 
import numpy as np 
import time 
import random
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.preprocessing import MinMaxScaler

In [2]:
class SeqDataLoader:  
    """序列数据迭代器"""
    def __init__(self, corpus, batch_size, num_steps, vocab_size):
        self.corpus = corpus
        self.batch_size, self.num_steps = batch_size, num_steps
        self.vocab_size = vocab_size

    def __iter__(self):
        return self.seq_data_iter_sequential(self.corpus, self.batch_size, self.num_steps, self.vocab_size)
    
    def seq_data_iter_sequential(self, corpus, batch_size, num_steps, vocab_size):
        """顺序生成一个小批量子序列"""
        # 从随机偏移量开始划分序列
        offset = random.randint(0, num_steps)
        num_tokens = ((len(corpus) - offset - 1) // batch_size) * batch_size
        Xs = torch.tensor(corpus[offset: offset + num_tokens], dtype=torch.float32)
        Ys = torch.tensor(corpus[offset + 1: offset + 1 + num_tokens], dtype=torch.float32)
        Xs, Ys = Xs.reshape(batch_size, -1, vocab_size), Ys.reshape(batch_size, -1, vocab_size)
        num_batches = Xs.shape[1] // num_steps
        for i in range(0, num_steps * num_batches, num_steps):
            X = Xs[:, i: i + num_steps]
            Y = Ys[:, i: i + num_steps][:, :, 0]
            yield X, Y

In [3]:
class RNNModel(nn.Module):
    """循环神经网络模型"""
    def __init__(self, rnn_layer, vocab_size, num_item, num_place, item_embed_size=5, place_embed_size=5, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)
        # item 和 place的embedding
        self.num_item = num_item
        self.num_place = num_place
        self.item_embed_size = item_embed_size
        self.place_embed_size = place_embed_size
        self.item_embed = nn.Embedding(num_embeddings=num_item, embedding_dim=item_embed_size)
        self.place_embed = nn.Embedding(num_embeddings=num_place, embedding_dim=place_embed_size)
        
    def forward(self, inputs, iid, pid, state):
        iid_tensor = torch.full((inputs.shape[0], inputs.shape[1], 1), iid)
        pid_tensor = torch.full((inputs.shape[0], inputs.shape[1], 1), pid)
        iid_embed = self.item_embed(iid_tensor).view(inputs.shape[0], inputs.shape[1], self.item_embed_size)
        pid_embed = self.place_embed(pid_tensor).view(inputs.shape[0], inputs.shape[1], self.place_embed_size)
        X = torch.cat((inputs, iid_embed, pid_embed), dim=-1)
        # 时间步数*批量大小,隐藏单元数
        X.transpose_(0, 1)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 将Y的形状改为(时间步数*批量大小,隐藏单元数)
        # 输出(时间步数*批量大小,词表大小)。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state

    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return  torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens), device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((self.num_directions * self.rnn.num_layers,
                batch_size, self.num_hiddens), device=device),
                    torch.zeros((self.num_directions * self.rnn.num_layers,
                        batch_size, self.num_hiddens), device=device))

In [4]:
# 这个函数也许有问题
def grad_clipping(net, theta):
    """裁剪梯度"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

def train_epoch(net, train_iter, iid, pid, loss, updater, device, use_random_iter):
    """训练网络一个迭代周期"""
    state = None
    start_time = time.time()
    mse = 0
    size = 0
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化state
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # state对于nn.GRU是个张量
                state.detach_()
            else:
                for s in state:
                    s.detach_()
                    
        # y = Y.reshape(-1)
        y = Y.transpose(0, 1).reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, iid, pid, state)
        l = loss(y_hat.reshape(-1), y)
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            updater(batch_size=1)
        mse += l * y.numel()
        size += y.numel()
    
    return mse / size, time.time() - start_time

def train(net, train_iter, iid, pid, lr, num_epochs, device, use_random_iter=False):
    """训练模型"""
    loss = nn.MSELoss()
    # 初始化
    updater = torch.optim.Adam(net.parameters(), lr)
    # 训练
    for epoch in range(num_epochs):
        epochmse, speed = train_epoch(
            net, train_iter, iid, pid, loss, updater, device, use_random_iter)
        if epoch == 0:
            print(f'epoch: {epoch + 1}, mse: {epochmse}, time: {speed}')
        if (epoch + 1) % 100 == 0:
            print(f'epoch: {epoch + 1}, mse: {epochmse}, time: {speed}')

In [5]:
def predict_rnn(inputs, iid, pid, num_preds, net, device):
    """向后预测"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = inputs.flatten().tolist() 
    # 预热
    Y, state = net(inputs, iid=iid, pid=pid, state=state)
    # 向后第一步
    outputs.append(Y.flatten()[-1].item())
    for _ in range(num_preds - 1):  # 预测num_preds步
        Y, state = net(torch.tensor(outputs[-1]).view(1, 1, 1), iid=iid, pid=pid, state=state)
        outputs.append(Y.item())
    return outputs

# test

In [68]:
import pandas as pd 

def gene(start_date, end_date, gid, pid):
    dr = pd.date_range(start_date, end_date)
    n = len(dr)
    qty = np.arange(n) / 50 #+ np.random.randint(-3, 3, n)
    df = pd.DataFrame({'date': dr, 
                      'goodsqty': qty,
                      'goodsid': gid,
                      'placepointid': pid})
    return df

# 根据goodsid， placepointid 获取对应的时间序列数据
def get_subdata(data, gid, pid):
    return data[(data['goodsid'] == gid) & (data['placepointid'] == pid)]
    
# 生成虚拟数据, 四个组合 (0, 0),(0, 1),(1, 0),(1, 1)
data = pd.concat((gene('1997-1-1', '1999-4-5', 0, 0), 
           gene('2020-1-1', '2022-3-3', 0, 1),
           gene('2015-12-12', '2018-9-9', 1, 0),
           gene('2025-6-9', '2027-9-9', 1, 1)), axis=0)

# 选出 goodsid = 0, placepointid = 0 的数据
subdata = get_subdata(data, 1, 0)

# 对日期排序
subdata = subdata.sort_values(by=['date'])

# 对subdata fit 一个ts模型 ......
# subdata

In [6]:
tsdata = pd.DataFrame({'sale': np.array([1,3,5,7,9,7,5,3] * 1000)})
# for i in range(1, 9):
#     tsdata['pre_{}'.format(i)] = tsdata['sale'].shift(i)
# tsdata.dropna(inplace=True)
iid = 0
pid = 0
data = tsdata.to_numpy()
scaler = MinMaxScaler() 
data = scaler.fit_transform(data)

In [7]:
# batch_size, num_steps = 64, 16
batch_size, num_steps = 64, 10
num_inputs = 1 + 3 + 3
num_hiddens = 16
net = RNNModel(nn.LSTM(num_inputs, num_hiddens), vocab_size=1, num_item=2, num_place=2, item_embed_size=3, place_embed_size=3)

In [8]:
for name, param in net.named_parameters():
    print(name, param.size())

rnn.weight_ih_l0 torch.Size([64, 7])
rnn.weight_hh_l0 torch.Size([64, 16])
rnn.bias_ih_l0 torch.Size([64])
rnn.bias_hh_l0 torch.Size([64])
linear.weight torch.Size([1, 16])
linear.bias torch.Size([1])
item_embed.weight torch.Size([2, 3])
place_embed.weight torch.Size([2, 3])


In [10]:
# 训练
train_iter = SeqDataLoader(data, batch_size=batch_size, num_steps=num_steps, vocab_size=1)
num_epochs, lr = 500, 0.01
device = 'cpu'

In [11]:
train(net, train_iter, iid, pid, lr, num_epochs, device)

epoch: 1, mse: 0.10659737139940262, time: 0.06602740287780762
epoch: 100, mse: 0.0010536934714764357, time: 0.06208324432373047
epoch: 200, mse: 0.0012371066259220243, time: 0.07005548477172852
epoch: 300, mse: 0.00035870165447704494, time: 0.06201481819152832
epoch: 400, mse: 2.6118932510144077e-05, time: 0.06301569938659668
epoch: 500, mse: 0.00046323001151904464, time: 0.06504106521606445


In [16]:
prefix = torch.tensor(data[-15:]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=iid, pid=pid, num_preds=15, net=net, device='cpu')

In [28]:
scaler.inverse_transform(np.array(predict).reshape(-1, 1))

array([[3.        ],
       [5.        ],
       [7.        ],
       [9.        ],
       [7.        ],
       [5.        ],
       [3.        ],
       [1.        ],
       [3.        ],
       [5.        ],
       [7.        ],
       [9.        ],
       [7.        ],
       [5.        ],
       [3.        ],
       [1.04147291],
       [3.04611945],
       [5.04017067],
       [7.05292225],
       [8.97053719],
       [6.97168303],
       [4.99179983],
       [2.96010947],
       [1.03233147],
       [3.06501389],
       [5.05541992],
       [7.07336569],
       [8.9685421 ],
       [6.95732021],
       [4.97966123]])

## linear

In [44]:
tsdata = pd.DataFrame({'sale': range(8000)})
# for i in range(1, 9):
#     tsdata['pre_{}'.format(i)] = tsdata['sale'].shift(i)
# tsdata.dropna(inplace=True)
iid = 0
pid = 0
data = tsdata.to_numpy()
scaler = MinMaxScaler() 
data = scaler.fit_transform(data)

# batch_size, num_steps = 64, 16
batch_size, num_steps = 64, 10
num_inputs = 1 + 3 + 3
num_hiddens = 16
net = RNNModel(nn.LSTM(num_inputs, num_hiddens), vocab_size=1, num_item=2, num_place=2, item_embed_size=3, place_embed_size=3)

# 训练
train_iter = SeqDataLoader(data, batch_size=batch_size, num_steps=num_steps, vocab_size=1)
num_epochs, lr = 500, 0.01
device = 'cpu'

In [45]:
train(net, train_iter, iid, pid, lr, num_epochs, device)

epoch: 1, mse: 0.25114333629608154, time: 0.06248593330383301
epoch: 100, mse: 9.349179890705273e-05, time: 0.06030607223510742
epoch: 200, mse: 4.446660568646621e-06, time: 0.07021689414978027
epoch: 300, mse: 2.5025333343364764e-06, time: 0.07030940055847168
epoch: 400, mse: 1.4038258086657152e-05, time: 0.07021641731262207
epoch: 500, mse: 5.091919956612401e-05, time: 0.060251474380493164


In [48]:
prefix = torch.tensor(data[-15:]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=iid, pid=pid, num_preds=15, net=net, device='cpu')
scaler.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([7985.        , 7986.        , 7987.        , 7988.        ,
       7989.        , 7990.        , 7991.        , 7992.        ,
       7993.        , 7994.        , 7995.        , 7996.        ,
       7997.        , 7998.        , 7999.        , 8042.66900969,
       8088.77053845, 8135.72359192, 8181.83656335, 8227.07321763,
       8271.50221074, 8315.04153693, 8357.65782177, 8399.33485484,
       8440.05928636, 8479.82062721, 8518.61506319, 8556.43687296,
       8593.28414941, 8629.15403187])

In [51]:
prefix = torch.tensor(data[10:30]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=iid, pid=pid, num_preds=15, net=net, device='cpu')
scaler.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([ 10.        ,  11.        ,  12.        ,  13.        ,
        14.        ,  15.        ,  16.        ,  17.        ,
        18.        ,  19.        ,  20.        ,  21.        ,
        22.        ,  23.        ,  24.        ,  25.        ,
        26.        ,  27.        ,  28.        ,  29.        ,
        69.65636621, 104.81024799, 138.6251    , 171.67377049,
       204.30287673, 236.64257899, 268.77345268, 300.75771728,
       332.64305054, 364.46687949, 396.25983709, 428.04898047,
       459.85767174, 491.70724668, 523.6188027 ])

# id 

In [16]:
def get_tsdata(data, gid, pid):
    return data[(data['goodsid'] == gid) & (data['placepointid'] == pid)]['goodsqty'].to_numpy()

In [21]:
scaler1 = MinMaxScaler() 
scaler2 = MinMaxScaler() 
scaler3 = MinMaxScaler() 

ts1 = np.array([1,3,5,7,9,7,5,3] * 1000)
ts2 = np.array([0,10,20,30,40,50,60,70,80,70,60,50,40,30,20,10] * 500)
ts3 = np.arange(8000)

df1 = pd.DataFrame({'goodsqty': scaler1.fit_transform(ts1.reshape(-1, 1)).reshape(-1),
                  'goodsid': 0,
                  'placepointid': 0})
df2 = pd.DataFrame({'goodsqty': scaler2.fit_transform(ts2.reshape(-1, 1)).reshape(-1),
                  'goodsid': 1,
                  'placepointid': 0})
df3 = pd.DataFrame({'goodsqty': scaler3.fit_transform(ts3.reshape(-1, 1)).reshape(-1),
                  'goodsid': 2,
                  'placepointid': 0})

data = pd.concat((df1, df2, df3), axis=0)

In [22]:
good_place = data[['goodsid', 'placepointid']].drop_duplicates().to_numpy()

In [27]:
# batch_size, num_steps = 64, 16
batch_size, num_steps = 64, 14
num_inputs = 1 + 3 + 3
num_hiddens = 16
net = RNNModel(nn.LSTM(num_inputs, num_hiddens), vocab_size=1, num_item=3, num_place=1, item_embed_size=3, place_embed_size=3)

In [28]:
# 训练
train_iter1 = SeqDataLoader(get_tsdata(data, 0, 0).reshape(-1, 1), batch_size=batch_size, num_steps=num_steps, vocab_size=1)
train_iter2 = SeqDataLoader(get_tsdata(data, 1, 0).reshape(-1, 1), batch_size=batch_size, num_steps=num_steps, vocab_size=1)
train_iter3 = SeqDataLoader(get_tsdata(data, 2, 0).reshape(-1, 1), batch_size=batch_size, num_steps=num_steps, vocab_size=1)
train_iter_lst = [(train_iter1, 0, 0), (train_iter2, 1, 0), (train_iter3, 2, 0)]
num_epochs, lr = 800, 0.01
device = 'cpu'

In [29]:
for _ in range(num_epochs):
    for train_iter, iid, pid in train_iter_lst:
        train(net, train_iter, iid, pid, lr, 1, device)

epoch: 1, mse: 0.10532869398593903, time: 0.06879854202270508
epoch: 1, mse: 0.07545756548643112, time: 0.04687905311584473
epoch: 1, mse: 0.03596240282058716, time: 0.0598757266998291
epoch: 1, mse: 0.09377843886613846, time: 0.05219078063964844
epoch: 1, mse: 0.047823794186115265, time: 0.05826163291931152
epoch: 1, mse: 0.006077420897781849, time: 0.05227398872375488
epoch: 1, mse: 0.06382707506418228, time: 0.06023836135864258
epoch: 1, mse: 0.03050898388028145, time: 0.05019664764404297
epoch: 1, mse: 0.00493152579292655, time: 0.05825066566467285
epoch: 1, mse: 0.04159929230809212, time: 0.050238847732543945
epoch: 1, mse: 0.01812414452433586, time: 0.058226823806762695
epoch: 1, mse: 0.005725582595914602, time: 0.05019831657409668
epoch: 1, mse: 0.025556471198797226, time: 0.0582432746887207
epoch: 1, mse: 0.01333511434495449, time: 0.05228900909423828
epoch: 1, mse: 0.0030036508105695248, time: 0.06017565727233887
epoch: 1, mse: 0.02947215549647808, time: 0.06025338172912598
ep

epoch: 1, mse: 0.0007108775316737592, time: 0.06021928787231445
epoch: 1, mse: 0.0016739965649321675, time: 0.050154924392700195
epoch: 1, mse: 0.0013385439524427056, time: 0.0521855354309082
epoch: 1, mse: 0.0005971972714178264, time: 0.0602269172668457
epoch: 1, mse: 0.0019251246703788638, time: 0.06030750274658203
epoch: 1, mse: 0.001022848067805171, time: 0.05824875831604004
epoch: 1, mse: 0.0010886897798627615, time: 0.05217576026916504
epoch: 1, mse: 0.0017056616488844156, time: 0.0581815242767334
epoch: 1, mse: 0.0012870950158685446, time: 0.05219411849975586
epoch: 1, mse: 0.0005946876481175423, time: 0.06027722358703613
epoch: 1, mse: 0.0016396658029407263, time: 0.060194969177246094
epoch: 1, mse: 0.0015007752226665616, time: 0.05016613006591797
epoch: 1, mse: 0.001035800902172923, time: 0.06026721000671387
epoch: 1, mse: 0.0017027712892740965, time: 0.05219292640686035
epoch: 1, mse: 0.0008848799043335021, time: 0.05814790725708008
epoch: 1, mse: 0.0005459717940539122, time:

epoch: 1, mse: 0.0009402887080796063, time: 0.06021475791931152
epoch: 1, mse: 0.0005047411541454494, time: 0.05222892761230469
epoch: 1, mse: 0.0005889979074709117, time: 0.0602874755859375
epoch: 1, mse: 0.0008903142297640443, time: 0.06023359298706055
epoch: 1, mse: 0.00040418971912004054, time: 0.06019306182861328
epoch: 1, mse: 0.0007122670649550855, time: 0.050215959548950195
epoch: 1, mse: 0.001028838800266385, time: 0.06027364730834961
epoch: 1, mse: 0.0006057401769794524, time: 0.050241708755493164
epoch: 1, mse: 0.0011073192581534386, time: 0.05223798751831055
epoch: 1, mse: 0.0006969450041651726, time: 0.060242652893066406
epoch: 1, mse: 0.0003935963904950768, time: 0.06029200553894043
epoch: 1, mse: 0.000794976600445807, time: 0.06023240089416504
epoch: 1, mse: 0.0010605714051052928, time: 0.0623323917388916
epoch: 1, mse: 0.0005505691515281796, time: 0.06018567085266113
epoch: 1, mse: 0.0006272537866607308, time: 0.05817294120788574
epoch: 1, mse: 0.0008417501230724156, ti

epoch: 1, mse: 0.00043266877764835954, time: 0.07020950317382812
epoch: 1, mse: 0.0008793336455710232, time: 0.05817103385925293
epoch: 1, mse: 0.0006441765581257641, time: 0.052207231521606445
epoch: 1, mse: 0.0003195078461430967, time: 0.060233116149902344
epoch: 1, mse: 0.0017403177917003632, time: 0.06024956703186035
epoch: 1, mse: 0.0006231156876310706, time: 0.05820822715759277
epoch: 1, mse: 0.0006104473141022027, time: 0.060320138931274414
epoch: 1, mse: 0.0015709084691479802, time: 0.060210227966308594
epoch: 1, mse: 0.0004942643572576344, time: 0.060254573822021484
epoch: 1, mse: 0.00015648454427719116, time: 0.050121307373046875
epoch: 1, mse: 0.0008914066711440682, time: 0.060189247131347656
epoch: 1, mse: 0.0005915262154303491, time: 0.0601651668548584
epoch: 1, mse: 0.00029739178717136383, time: 0.06022977828979492
epoch: 1, mse: 0.000983520527370274, time: 0.060194969177246094
epoch: 1, mse: 0.0006214701570570469, time: 0.0602717399597168
epoch: 1, mse: 0.000255720486165

epoch: 1, mse: 0.0006067714421078563, time: 0.05220198631286621
epoch: 1, mse: 0.0002953644434455782, time: 0.05224037170410156
epoch: 1, mse: 0.0007486645481549203, time: 0.06025409698486328
epoch: 1, mse: 0.00043284514686092734, time: 0.06023716926574707
epoch: 1, mse: 0.00027618909371085465, time: 0.07031702995300293
epoch: 1, mse: 0.0017368311528116465, time: 0.0602262020111084
epoch: 1, mse: 0.0006194954621605575, time: 0.06015443801879883
epoch: 1, mse: 0.0004499316564761102, time: 0.05014657974243164
epoch: 1, mse: 0.0013392956461757421, time: 0.05215930938720703
epoch: 1, mse: 0.0005679834284819663, time: 0.052220821380615234
epoch: 1, mse: 0.0004137811774853617, time: 0.06028461456298828
epoch: 1, mse: 0.0011227737413719296, time: 0.06028175354003906
epoch: 1, mse: 0.00046810577623546124, time: 0.058183908462524414
epoch: 1, mse: 0.0002670280518941581, time: 0.05014491081237793
epoch: 1, mse: 0.0004922105581499636, time: 0.0602107048034668
epoch: 1, mse: 0.0006188602419570088,

...

epoch: 1, mse: 0.0002470176259521395, time: 0.05017447471618652
epoch: 1, mse: 0.0008397593046538532, time: 0.05221295356750488
epoch: 1, mse: 0.00019722638535313308, time: 0.05218195915222168
epoch: 1, mse: 0.00016089074779301882, time: 0.06018495559692383
epoch: 1, mse: 0.0004271713551133871, time: 0.06019878387451172
epoch: 1, mse: 0.00023607532784808427, time: 0.060161590576171875
epoch: 1, mse: 0.0002078310353681445, time: 0.050131797790527344
epoch: 1, mse: 0.0008161842706613243, time: 0.05215787887573242
epoch: 1, mse: 0.00027653566212393343, time: 0.06018376350402832
epoch: 1, mse: 0.00020580715499818325, time: 0.060172080993652344
epoch: 1, mse: 0.00031620488152839243, time: 0.050215959548950195
epoch: 1, mse: 0.00032773506245575845, time: 0.06238436698913574
epoch: 1, mse: 0.00011469919991213828, time: 0.05825614929199219
epoch: 1, mse: 0.0008792329463176429, time: 0.05222654342651367
epoch: 1, mse: 0.0002863427216652781, time: 0.060280799865722656
epoch: 1, mse: 0.0002701868

epoch: 1, mse: 0.0008442825055681169, time: 0.06037282943725586
epoch: 1, mse: 0.00031023964402265847, time: 0.06029176712036133
epoch: 1, mse: 0.00016688807227183133, time: 0.060189247131347656
epoch: 1, mse: 0.0007960119983181357, time: 0.050202131271362305
epoch: 1, mse: 0.0002698523458093405, time: 0.05219459533691406
epoch: 1, mse: 0.0001983225520234555, time: 0.060272216796875
epoch: 1, mse: 0.000768563651945442, time: 0.050176143646240234
epoch: 1, mse: 0.00024094375839922577, time: 0.06020188331604004
epoch: 1, mse: 0.0002501989365555346, time: 0.05822014808654785
epoch: 1, mse: 0.00025712238857522607, time: 0.052201032638549805
epoch: 1, mse: 0.0002575713151600212, time: 0.060202598571777344
epoch: 1, mse: 0.00017195248801726848, time: 0.05819439888000488
epoch: 1, mse: 0.0002379618672421202, time: 0.05022716522216797
epoch: 1, mse: 0.0004097523051314056, time: 0.06226491928100586
epoch: 1, mse: 0.0001649588521104306, time: 0.05831170082092285
epoch: 1, mse: 0.0007102337549440

epoch: 1, mse: 9.949801460606977e-05, time: 0.052266836166381836
epoch: 1, mse: 0.0009667716803960502, time: 0.06026315689086914
epoch: 1, mse: 0.0002805170661304146, time: 0.050214529037475586
epoch: 1, mse: 0.00022855518909636885, time: 0.05224323272705078
epoch: 1, mse: 0.0008175935363397002, time: 0.06051802635192871
epoch: 1, mse: 0.0002504306030459702, time: 0.05024552345275879
epoch: 1, mse: 0.00015249101852532476, time: 0.06031537055969238
epoch: 1, mse: 0.0008478265954181552, time: 0.06023907661437988
epoch: 1, mse: 0.00021161430049687624, time: 0.060233354568481445
epoch: 1, mse: 8.496585360262543e-05, time: 0.05817365646362305
epoch: 1, mse: 0.00027074600802734494, time: 0.05215787887573242
epoch: 1, mse: 0.00030106803751550615, time: 0.06014823913574219
epoch: 1, mse: 0.00018258941418025643, time: 0.05822277069091797
epoch: 1, mse: 0.0007439896580763161, time: 0.052201271057128906
epoch: 1, mse: 0.0002669558161869645, time: 0.0601963996887207
epoch: 1, mse: 0.00017119171388

epoch: 1, mse: 0.00042075529927387834, time: 0.06026101112365723
epoch: 1, mse: 0.00017638500139582902, time: 0.058232784271240234
epoch: 1, mse: 0.00023485990823246539, time: 0.05025362968444824
epoch: 1, mse: 0.0002275475999340415, time: 0.06028604507446289
epoch: 1, mse: 0.0001933894818648696, time: 0.06025958061218262
epoch: 1, mse: 0.001066644792445004, time: 0.060187578201293945
epoch: 1, mse: 0.0002329456910956651, time: 0.060257673263549805
epoch: 1, mse: 0.00019836894352920353, time: 0.05817294120788574
epoch: 1, mse: 0.00023213688109535724, time: 0.05821847915649414
epoch: 1, mse: 0.0002797469205688685, time: 0.050158023834228516
epoch: 1, mse: 0.0002365281543461606, time: 0.06019020080566406
epoch: 1, mse: 0.0006457066047005355, time: 0.05815577507019043
epoch: 1, mse: 0.00022171683667693287, time: 0.05219078063964844
epoch: 1, mse: 0.0002152818051399663, time: 0.060176849365234375
epoch: 1, mse: 0.000562526227440685, time: 0.050130605697631836
epoch: 1, mse: 0.0002014632045

epoch: 1, mse: 0.0009244423708878458, time: 0.05019569396972656
epoch: 1, mse: 0.0003111720143351704, time: 0.052197933197021484
epoch: 1, mse: 0.00010526907863095403, time: 0.060193777084350586
epoch: 1, mse: 0.0002455454377923161, time: 0.05014657974243164
epoch: 1, mse: 0.00027255434542894363, time: 0.06020379066467285
epoch: 1, mse: 0.00027311331359669566, time: 0.06027626991271973
epoch: 1, mse: 0.0007080739596858621, time: 0.05018162727355957
epoch: 1, mse: 0.00035426532849669456, time: 0.05216073989868164
epoch: 1, mse: 0.0001544385013403371, time: 0.06028914451599121
epoch: 1, mse: 0.00024778256192803383, time: 0.058205604553222656
epoch: 1, mse: 0.0004194331413600594, time: 0.050344228744506836
epoch: 1, mse: 0.00013658308307640254, time: 0.0602717399597168
epoch: 1, mse: 0.0005991215584799647, time: 0.06030082702636719
epoch: 1, mse: 0.0002633249678183347, time: 0.05015897750854492
epoch: 1, mse: 0.00014560125418938696, time: 0.05217266082763672
epoch: 1, mse: 0.0007707466138

# two ts

## 0，0 

In [123]:
prefix = torch.tensor(df1['goodsqty'].values[-15:]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=0, pid=0, num_preds=15, net=net, device='cpu')
scaler1.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([3.        , 5.        , 7.        , 9.        , 7.        ,
       5.        , 3.        , 1.        , 3.        , 5.        ,
       7.        , 9.        , 7.        , 5.        , 3.        ,
       1.04882205, 3.00356889, 5.10103893, 7.32742453, 9.39713955,
       7.22058725, 5.22507668, 3.25273657, 1.16065454, 2.75694323,
       4.89660287, 7.02970886, 9.44234657, 7.32826614, 5.33908081])

## 1， 0

In [124]:
prefix = torch.tensor(df2['goodsqty'].values[-15:]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=1, pid=0, num_preds=15, net=net, device='cpu')
scaler2.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([10.        , 20.        , 30.        , 40.        , 50.        ,
       60.        , 70.        , 80.        , 70.        , 60.        ,
       50.        , 40.        , 30.        , 20.        , 10.        ,
        0.64081192, 10.62502742, 21.41913891, 32.23875999, 43.53314877,
       54.72894669, 65.50070763, 76.7827034 , 79.10337448, 67.65916824,
       57.75594234, 47.98781872, 39.14516449, 30.6065011 , 23.29094648])

In [125]:
prefix = torch.tensor(df2['goodsqty'].values[-15:]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=0, pid=0, num_preds=15, net=net, device='cpu')
scaler2.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([10.        , 20.        , 30.        , 40.        , 50.        ,
       60.        , 70.        , 80.        , 70.        , 60.        ,
       50.        , 40.        , 30.        , 20.        , 10.        ,
       21.50633335, 39.84955549, 62.35590458, 84.48865891, 62.60575771,
       42.61460304, 22.94732094,  1.87743068, 17.10181713, 38.56281042,
       59.72314358, 84.32671547, 63.45131874, 43.57002258, 24.03068542])

# three ts

## 0， 0

In [38]:
prefix = torch.tensor(df1['goodsqty'].values[-15:]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=0, pid=0, num_preds=15, net=net, device='cpu')
scaler1.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([3.        , 5.        , 7.        , 9.        , 7.        ,
       5.        , 3.        , 1.        , 3.        , 5.        ,
       7.        , 9.        , 7.        , 5.        , 3.        ,
       0.87796736, 2.91351712, 4.90321541, 6.80277157, 8.83882236,
       7.15236092, 5.22350025, 3.37243104, 1.13308334, 2.79449439,
       4.75773144, 6.73889351, 8.81169224, 7.21612644, 5.27567053])

## 1，0

In [37]:
prefix = torch.tensor(df2['goodsqty'].values[-15:]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=1, pid=0, num_preds=15, net=net, device='cpu')
scaler2.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([10.        , 20.        , 30.        , 40.        , 50.        ,
       60.        , 70.        , 80.        , 70.        , 60.        ,
       50.        , 40.        , 30.        , 20.        , 10.        ,
       -0.50688744,  9.65742111, 19.69857454, 29.40921068, 39.12338734,
       48.03080082, 56.0827589 , 63.62910748, 73.75700951, 73.73124123,
       63.5524416 , 53.30701828, 42.83133984, 32.09264278, 20.33379078])

## 2，0

In [42]:
prefix = torch.tensor(df3['goodsqty'].values[-25:]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=2, pid=0, num_preds=15, net=net, device='cpu')
scaler3.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([7975.        , 7976.        , 7977.        , 7978.        ,
       7979.        , 7980.        , 7981.        , 7982.        ,
       7983.        , 7984.        , 7985.        , 7986.        ,
       7987.        , 7988.        , 7989.        , 7990.        ,
       7991.        , 7992.        , 7993.        , 7994.        ,
       7995.        , 7996.        , 7997.        , 7998.        ,
       7999.        , 7790.44891608, 7671.03711712, 7553.29403961,
       7452.37358129, 7354.07730663, 7257.44403207, 7161.38861191,
       7066.14752781, 6971.81327462, 6878.53556049, 6786.41355515,
       6695.53879988, 6605.98281133, 6517.8070938 , 6431.05360371])

In [43]:
prefix = torch.tensor(df3['goodsqty'].values[10:30]).reshape(1, -1, 1)
# 向后预测
predict = predict_rnn(inputs=prefix, iid=2, pid=0, num_preds=15, net=net, device='cpu')
scaler3.inverse_transform(np.array(predict).reshape(-1, 1)).reshape(-1)

array([  10.        ,   11.        ,   12.        ,   13.        ,
         14.        ,   15.        ,   16.        ,   17.        ,
         18.        ,   19.        ,   20.        ,   21.        ,
         22.        ,   23.        ,   24.        ,   25.        ,
         26.        ,   27.        ,   28.        ,   29.        ,
        -15.86381954,  -49.79607803,  -88.39336649, -125.29332685,
       -155.81376517, -184.93486136, -214.1124557 , -242.29167783,
       -269.86849153, -297.24362832, -324.20659092, -350.80505708,
       -377.13771975, -403.16214573, -428.86951464])